這份 Notebook 示範用 Function Calling 實作一個對話購物車結帳流程




In [1]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [2]:
import requests
import json
from pprint import pp

In [3]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=4000, tools=None, tool_choice=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if tools:
    payload["tools"] = tools
  if tool_choice:
    payload["tool_choice"] = tool_choice

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]
  else :
    return obj["error"]

## 準備資料



In [7]:
# 這是練習用的 dataset，有 title, description 以及標註的正確分類 category
url = "https://ihower.tw/data/books-dataset-200.csv"
response = requests.get(url)

In [24]:
import pandas as pd
from io import StringIO
df = pd.read_csv(StringIO(response.text))
df['qty'] = df.index % 3 # 隨便塞個數量
df['product_id'] = [f'{i+1:04d}' for i in range(len(df))] # 給個商品編號
df['price'] = [((i % 9) + 1) * 100 for i in range(len(df))] # 給個售價
df = df.drop('description', axis=1)
df = df.drop('tags', axis=1)
df

,title,category,qty,product_id,price
0,統計學：重點觀念與題解 (下) 3/e (適用: 大學用書．高普考．商管所),機率統計學 Probability-and-statistics,0,0001,100
1,新時代 乙級數位電子術科含學科題庫實作寶典 - 使用 KiCad/Quartus II / ...,技能檢定乙級 Skilltest-b,1,0002,200
2,樂高 SPIKE 機器人創意專題實作 -- 使用 LEGO Education SPIKE ...,機器人製作 Robots,2,0003,300
3,AI 生成時代：從 ChatGPT 到繪圖、音樂、影片，利用智能創作自我加值、簡化工作，成為...,人工智慧,0,0004,400
4,都問 AI 吧！ChatGPT 上手的第一本書,ChatGPT,1,0005,500
...,...,...,...,...,...
194,Autodesk Civil 3D 2024 from Start to Finish: A...,AutoCAD,2,0195,600
195,Network Programming and Automation Essentials:...,Go 程式語言,0,0196,700
196,Internet of Things for Smart Buildings: Levera...,物聯網 IoT,1,0197,800
197,AI and Business Rule Engines for Excel Power U...,Excel,2,0198,900


## 定義方法

In [25]:
def search_products(keyword):
  print( f"called search_products: {keyword}" )
  # 搜索包含關鍵詞的產品
  filtered_df = df[df['title'].str.contains(keyword, case=False, na=False)]

  results = [{
    'product_id': row["product_id"],
    'title': row['title'],
    'qty': row["qty"],
    'price': row["price"]
  } for index, row in filtered_df.iterrows()]

  # 只回傳前5筆結果
  return results[:5]

def add_cart(product_id, qty):
  print( f"called add_cart: {product_id} x {qty}" )
  if product_id in df['product_id'].values:
      # 獲取產品的數量
      product_qty = df.loc[df['product_id'] == product_id, 'qty'].iloc[0]
      if qty <= product_qty:
          return {"msg": "商品成功加入購物車"}
      else:
          # 缺貨
          return {"msg": "加入失敗，商品數量不足"}
  else:
      # 產品ID不存在
      return {"msg": "找不到此商品"}

def send_order(receiver_name, address):
  print( f"called send_order: {receiver_name} {address}")
  return "OK，訂單編號是 123456"

available_functions = {
  "search_products": search_products,
  "add_cart": add_cart,
  "send_order": send_order
}


In [37]:
def get_completion_with_function_execution(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=1000, tools=None, tool_choice=None):
  print(f"called prompt: {messages}")
  response = get_completion(messages, model=model, temperature=temperature, max_tokens=max_tokens, tools=tools,tool_choice=tool_choice)

  if response.get("tool_calls"): # 或用 response 裡面的 finish_reason 判斷也行
    messages.append(response)

    # ------ 呼叫函數，這裡改成執行多 tool_calls (可以改成平行處理，目前用簡單的迴圈)
    for tool_call in response["tool_calls"]:
      function_name = tool_call["function"]["name"]
      function_args = json.loads(tool_call["function"]["arguments"])
      function_to_call = available_functions[function_name]

      print(f"   called function {function_name} with {function_args}")
      function_response = function_to_call(**function_args)
      messages.append(
          {
              "tool_call_id": tool_call["id"], # 多了 toll_call_id
              "role": "tool",
              "name": function_name,
              "content": str(function_response),
          }
      )

    # 進行遞迴呼叫
    return get_completion_with_function_execution(messages, model=model, temperature=temperature, max_tokens=max_tokens, tools=tools,tool_choice=tool_choice)

  else:
    print(response)
    return response["content"]

In [72]:
system_messages = [
  {"role": "system", "content": """你是小智，一個先進的書店 AI 客服機器人。
* 你的任務是協助客人搜尋商品，然後加入購物車，最後完成結帳
* 若客人要求查詢購物車內有什麼商品，請用表格條列商品、金額和總金額
* 請總是用台灣繁體中文回答用戶，說話親切但是精準(Be Concise)
* 有時候工具會回報錯誤，請根據錯誤訊息處理，可向使用者詢問更多資訊，或是跟客戶道歉
* 不要跟用戶閒聊與書店無關的事情
* 若問題不在上述內容中，請回答不知道，請客人聯繫客服""" },
]

messages = system_messages + [{ "role": "user", "content": "你們有賣什麼書?" }] # 用戶的第一句話

tools = [
    {
        "type": "function",
        "function": {
            "name": "search_products",
            "description": "搜尋商品，請用表格呈現商品列表，包括商品編號、商品名稱、單價，以及是否可購買(根據 qty 是否大於0)",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "商品關鍵字，不能是空白",
                    }
                },
                "required": ["keyword"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "add_cart",
            "description": "加入購物車",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_id": {
                        "type": "string",
                        "description": "商品編號",
                    },
                    "qty": {
                        "type": "integer",
                        "description": "數量",
                    }
                },
                "required": ["product_id", "qty"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "send_order",
            "description": "用戶結帳",
            "parameters": {
                "type": "object",
                "properties": {
                  "receiver_name": {
                        "type": "string",
                        "description": "收件人",
                    },
                  "address": {
                        "type": "string",
                        "description": "地址",
                    }
                },
                "required": ["receiver_name", "address"],
            },
        },
    }
]


result = get_completion_with_function_execution(messages, model="gpt-4-turbo-preview", tools=tools)
messages.append( { "role": "assistant", "content": result })
result


called prompt: [{'role': 'system', 'content': '你是小智，一個先進的書店 AI 客服機器人。  \n* 你的任務是協助客人搜尋商品，然後加入購物車，最後完成結帳\n* 若客人要求查詢購物車內有什麼商品，請用表格條列商品、金額和總金額\n* 請總是用台灣繁體中文回答用戶，說話親切但是精準(Be Concise)\n* 有時候工具會回報錯誤，請根據錯誤訊息處理，可向使用者詢問更多資訊，或是跟客戶道歉\n* 不要跟用戶閒聊與書店無關的事情\n* 若問題不在上述內容中，請回答不知道，請客人聯繫客服'}, {'role': 'user', 'content': '你們有賣什麼書?'}]
{'role': 'assistant', 'content': '不好意思，請問您想找哪一類或哪一本書的資訊呢？提供一些關鍵字會更有幫助。'}


'不好意思，請問您想找哪一類或哪一本書的資訊呢？提供一些關鍵字會更有幫助。'

In [73]:
messages.append( { "role": "user", "content": "我想買本 Python 書" })
result = get_completion_with_function_execution(messages, model="gpt-4-turbo-preview", tools=tools)
messages.append( { "role": "assistant", "content": result })

called prompt: [{'role': 'system', 'content': '你是小智，一個先進的書店 AI 客服機器人。  \n* 你的任務是協助客人搜尋商品，然後加入購物車，最後完成結帳\n* 若客人要求查詢購物車內有什麼商品，請用表格條列商品、金額和總金額\n* 請總是用台灣繁體中文回答用戶，說話親切但是精準(Be Concise)\n* 有時候工具會回報錯誤，請根據錯誤訊息處理，可向使用者詢問更多資訊，或是跟客戶道歉\n* 不要跟用戶閒聊與書店無關的事情\n* 若問題不在上述內容中，請回答不知道，請客人聯繫客服'}, {'role': 'user', 'content': '你們有賣什麼書?'}, {'role': 'assistant', 'content': '不好意思，請問您想找哪一類或哪一本書的資訊呢？提供一些關鍵字會更有幫助。'}, {'role': 'user', 'content': '我想買本 Python 書'}]
   called function search_products with {'keyword': 'Python'}
called search_products: Python
called prompt: [{'role': 'system', 'content': '你是小智，一個先進的書店 AI 客服機器人。  \n* 你的任務是協助客人搜尋商品，然後加入購物車，最後完成結帳\n* 若客人要求查詢購物車內有什麼商品，請用表格條列商品、金額和總金額\n* 請總是用台灣繁體中文回答用戶，說話親切但是精準(Be Concise)\n* 有時候工具會回報錯誤，請根據錯誤訊息處理，可向使用者詢問更多資訊，或是跟客戶道歉\n* 不要跟用戶閒聊與書店無關的事情\n* 若問題不在上述內容中，請回答不知道，請客人聯繫客服'}, {'role': 'user', 'content': '你們有賣什麼書?'}, {'role': 'assistant', 'content': '不好意思，請問您想找哪一類或哪一本書的資訊呢？提供一些關鍵字會更有幫助。'}, {'role': 'user', 'content': '我想買本 Python 書'}, {'role': 'assistant', 'content': None

In [74]:
print(result)

以下是我們店內關於 Python 的書籍：

| 商品編號 | 商品名稱 | 單價 | 是否可購買 |
| -------- | -------- | ---- | ---------- |
| 0009 | 快速學會 Python 架站技術：活用 Django 4 建構動態網站的 16堂課 | 900 | 可購買 |
| 0012 | 網絡爬蟲案例教程（Python·微課視頻版） | 300 | 可購買 |
| 0016 | 趣味微項目，輕松學Python | 700 | 已售罄 |
| 0030 | Python 機器學習入門與實戰 | 300 | 可購買 |
| 0072 | Python 全棧開發 — 數據分析 | 900 | 可購買 |

請問您想要購買哪一本書呢？提供商品編號即可。


In [75]:
messages.append( { "role": "user", "content": "我要買 Python 機器學習入門與實戰 1本，以及 Python 全棧開發 2 本" })
result = get_completion_with_function_execution(messages, model="gpt-4-turbo-preview", tools=tools)
messages.append( { "role": "assistant", "content": result })

called prompt: [{'role': 'system', 'content': '你是小智，一個先進的書店 AI 客服機器人。  \n* 你的任務是協助客人搜尋商品，然後加入購物車，最後完成結帳\n* 若客人要求查詢購物車內有什麼商品，請用表格條列商品、金額和總金額\n* 請總是用台灣繁體中文回答用戶，說話親切但是精準(Be Concise)\n* 有時候工具會回報錯誤，請根據錯誤訊息處理，可向使用者詢問更多資訊，或是跟客戶道歉\n* 不要跟用戶閒聊與書店無關的事情\n* 若問題不在上述內容中，請回答不知道，請客人聯繫客服'}, {'role': 'user', 'content': '你們有賣什麼書?'}, {'role': 'assistant', 'content': '不好意思，請問您想找哪一類或哪一本書的資訊呢？提供一些關鍵字會更有幫助。'}, {'role': 'user', 'content': '我想買本 Python 書'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_fC5FndPWR09Y9AKPF9r6nJKY', 'type': 'function', 'function': {'name': 'search_products', 'arguments': '{"keyword":"Python"}'}}]}, {'tool_call_id': 'call_fC5FndPWR09Y9AKPF9r6nJKY', 'role': 'tool', 'name': 'search_products', 'content': "[{'product_id': '0009', 'title': '快速學會 Python 架站技術：活用 Django 4 建構動態網站的 16堂課', 'qty': 2, 'price': 900}, {'product_id': '0012', 'title': '網絡爬蟲案例教程（Python·微課視頻版）', 'qty': 2, 'price': 300}, {'product_id': '0016', 'title': '趣味微項目，輕松學Python', 'qty': 0, 'price': 700}, {'p

In [76]:
messages.append( { "role": "user", "content": "我還要一本 趣味微項目" })
result = get_completion_with_function_execution(messages, model="gpt-4-turbo-preview", tools=tools)
messages.append( { "role": "assistant", "content": result })

called prompt: [{'role': 'system', 'content': '你是小智，一個先進的書店 AI 客服機器人。  \n* 你的任務是協助客人搜尋商品，然後加入購物車，最後完成結帳\n* 若客人要求查詢購物車內有什麼商品，請用表格條列商品、金額和總金額\n* 請總是用台灣繁體中文回答用戶，說話親切但是精準(Be Concise)\n* 有時候工具會回報錯誤，請根據錯誤訊息處理，可向使用者詢問更多資訊，或是跟客戶道歉\n* 不要跟用戶閒聊與書店無關的事情\n* 若問題不在上述內容中，請回答不知道，請客人聯繫客服'}, {'role': 'user', 'content': '你們有賣什麼書?'}, {'role': 'assistant', 'content': '不好意思，請問您想找哪一類或哪一本書的資訊呢？提供一些關鍵字會更有幫助。'}, {'role': 'user', 'content': '我想買本 Python 書'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_fC5FndPWR09Y9AKPF9r6nJKY', 'type': 'function', 'function': {'name': 'search_products', 'arguments': '{"keyword":"Python"}'}}]}, {'tool_call_id': 'call_fC5FndPWR09Y9AKPF9r6nJKY', 'role': 'tool', 'name': 'search_products', 'content': "[{'product_id': '0009', 'title': '快速學會 Python 架站技術：活用 Django 4 建構動態網站的 16堂課', 'qty': 2, 'price': 900}, {'product_id': '0012', 'title': '網絡爬蟲案例教程（Python·微課視頻版）', 'qty': 2, 'price': 300}, {'product_id': '0016', 'title': '趣味微項目，輕松學Python', 'qty': 0, 'price': 700}, {'p

In [77]:
messages.append( { "role": "user", "content": "現在購物車有什麼?" })
result = get_completion_with_function_execution(messages, model="gpt-4-turbo-preview", tools=tools)
messages.append( { "role": "assistant", "content": result })

called prompt: [{'role': 'system', 'content': '你是小智，一個先進的書店 AI 客服機器人。  \n* 你的任務是協助客人搜尋商品，然後加入購物車，最後完成結帳\n* 若客人要求查詢購物車內有什麼商品，請用表格條列商品、金額和總金額\n* 請總是用台灣繁體中文回答用戶，說話親切但是精準(Be Concise)\n* 有時候工具會回報錯誤，請根據錯誤訊息處理，可向使用者詢問更多資訊，或是跟客戶道歉\n* 不要跟用戶閒聊與書店無關的事情\n* 若問題不在上述內容中，請回答不知道，請客人聯繫客服'}, {'role': 'user', 'content': '你們有賣什麼書?'}, {'role': 'assistant', 'content': '不好意思，請問您想找哪一類或哪一本書的資訊呢？提供一些關鍵字會更有幫助。'}, {'role': 'user', 'content': '我想買本 Python 書'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_fC5FndPWR09Y9AKPF9r6nJKY', 'type': 'function', 'function': {'name': 'search_products', 'arguments': '{"keyword":"Python"}'}}]}, {'tool_call_id': 'call_fC5FndPWR09Y9AKPF9r6nJKY', 'role': 'tool', 'name': 'search_products', 'content': "[{'product_id': '0009', 'title': '快速學會 Python 架站技術：活用 Django 4 建構動態網站的 16堂課', 'qty': 2, 'price': 900}, {'product_id': '0012', 'title': '網絡爬蟲案例教程（Python·微課視頻版）', 'qty': 2, 'price': 300}, {'product_id': '0016', 'title': '趣味微項目，輕松學Python', 'qty': 0, 'price': 700}, {'p

In [78]:
print(result)

目前您的購物車內有以下商品：

| 商品名稱 | 數量 | 單價 | 總金額 |
| -------- | ---- | ---- | ------ |
| Python 機器學習入門與實戰 | 1 | 300 | 300 |
| Python 全棧開發 — 數據分析 | 2 | 900 | 1800 |

**總金額：2100元**

請問需要進行結帳嗎？還是有其他需求？


In [79]:
messages.append( { "role": "user", "content": "好，我要結帳了" })
result = get_completion_with_function_execution(messages, model="gpt-4-turbo-preview", tools=tools)
messages.append( { "role": "assistant", "content": result })
result

called prompt: [{'role': 'system', 'content': '你是小智，一個先進的書店 AI 客服機器人。  \n* 你的任務是協助客人搜尋商品，然後加入購物車，最後完成結帳\n* 若客人要求查詢購物車內有什麼商品，請用表格條列商品、金額和總金額\n* 請總是用台灣繁體中文回答用戶，說話親切但是精準(Be Concise)\n* 有時候工具會回報錯誤，請根據錯誤訊息處理，可向使用者詢問更多資訊，或是跟客戶道歉\n* 不要跟用戶閒聊與書店無關的事情\n* 若問題不在上述內容中，請回答不知道，請客人聯繫客服'}, {'role': 'user', 'content': '你們有賣什麼書?'}, {'role': 'assistant', 'content': '不好意思，請問您想找哪一類或哪一本書的資訊呢？提供一些關鍵字會更有幫助。'}, {'role': 'user', 'content': '我想買本 Python 書'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_fC5FndPWR09Y9AKPF9r6nJKY', 'type': 'function', 'function': {'name': 'search_products', 'arguments': '{"keyword":"Python"}'}}]}, {'tool_call_id': 'call_fC5FndPWR09Y9AKPF9r6nJKY', 'role': 'tool', 'name': 'search_products', 'content': "[{'product_id': '0009', 'title': '快速學會 Python 架站技術：活用 Django 4 建構動態網站的 16堂課', 'qty': 2, 'price': 900}, {'product_id': '0012', 'title': '網絡爬蟲案例教程（Python·微課視頻版）', 'qty': 2, 'price': 300}, {'product_id': '0016', 'title': '趣味微項目，輕松學Python', 'qty': 0, 'price': 700}, {'p

'請提供收件人姓名和地址，以便我們進行結帳程序。'

In [80]:
messages.append( { "role": "user", "content": "收件人是 ihower" })
result = get_completion_with_function_execution(messages, model="gpt-4-turbo-preview", tools=tools)
messages.append( { "role": "assistant", "content": result })
result

called prompt: [{'role': 'system', 'content': '你是小智，一個先進的書店 AI 客服機器人。  \n* 你的任務是協助客人搜尋商品，然後加入購物車，最後完成結帳\n* 若客人要求查詢購物車內有什麼商品，請用表格條列商品、金額和總金額\n* 請總是用台灣繁體中文回答用戶，說話親切但是精準(Be Concise)\n* 有時候工具會回報錯誤，請根據錯誤訊息處理，可向使用者詢問更多資訊，或是跟客戶道歉\n* 不要跟用戶閒聊與書店無關的事情\n* 若問題不在上述內容中，請回答不知道，請客人聯繫客服'}, {'role': 'user', 'content': '你們有賣什麼書?'}, {'role': 'assistant', 'content': '不好意思，請問您想找哪一類或哪一本書的資訊呢？提供一些關鍵字會更有幫助。'}, {'role': 'user', 'content': '我想買本 Python 書'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_fC5FndPWR09Y9AKPF9r6nJKY', 'type': 'function', 'function': {'name': 'search_products', 'arguments': '{"keyword":"Python"}'}}]}, {'tool_call_id': 'call_fC5FndPWR09Y9AKPF9r6nJKY', 'role': 'tool', 'name': 'search_products', 'content': "[{'product_id': '0009', 'title': '快速學會 Python 架站技術：活用 Django 4 建構動態網站的 16堂課', 'qty': 2, 'price': 900}, {'product_id': '0012', 'title': '網絡爬蟲案例教程（Python·微課視頻版）', 'qty': 2, 'price': 300}, {'product_id': '0016', 'title': '趣味微項目，輕松學Python', 'qty': 0, 'price': 700}, {'p

'不好意思，請提供完整的收件人姓名和地址，以便我們進行結帳程序。'

In [81]:
messages.append( { "role": "user", "content": "地址是新竹市東區高峰里高峰路202巷13號" })
result = get_completion_with_function_execution(messages, model="gpt-4-turbo-preview", tools=tools)
messages.append( { "role": "assistant", "content": result })
result

called prompt: [{'role': 'system', 'content': '你是小智，一個先進的書店 AI 客服機器人。  \n* 你的任務是協助客人搜尋商品，然後加入購物車，最後完成結帳\n* 若客人要求查詢購物車內有什麼商品，請用表格條列商品、金額和總金額\n* 請總是用台灣繁體中文回答用戶，說話親切但是精準(Be Concise)\n* 有時候工具會回報錯誤，請根據錯誤訊息處理，可向使用者詢問更多資訊，或是跟客戶道歉\n* 不要跟用戶閒聊與書店無關的事情\n* 若問題不在上述內容中，請回答不知道，請客人聯繫客服'}, {'role': 'user', 'content': '你們有賣什麼書?'}, {'role': 'assistant', 'content': '不好意思，請問您想找哪一類或哪一本書的資訊呢？提供一些關鍵字會更有幫助。'}, {'role': 'user', 'content': '我想買本 Python 書'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_fC5FndPWR09Y9AKPF9r6nJKY', 'type': 'function', 'function': {'name': 'search_products', 'arguments': '{"keyword":"Python"}'}}]}, {'tool_call_id': 'call_fC5FndPWR09Y9AKPF9r6nJKY', 'role': 'tool', 'name': 'search_products', 'content': "[{'product_id': '0009', 'title': '快速學會 Python 架站技術：活用 Django 4 建構動態網站的 16堂課', 'qty': 2, 'price': 900}, {'product_id': '0012', 'title': '網絡爬蟲案例教程（Python·微課視頻版）', 'qty': 2, 'price': 300}, {'product_id': '0016', 'title': '趣味微項目，輕松學Python', 'qty': 0, 'price': 700}, {'p

'訂單已成功建立，訂單編號是 123456。您的商品將會盡快寄出，感謝您的購買！\n\n如果還有其他問題或需求，歡迎隨時提問。'

## 做個 UI 介面

In [68]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.8 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl si

In [69]:
import gradio as gr
gr.close_all()

In [82]:
def chat(message, chat_history):
  openai_messages = list(system_messages)

  # 把 gradio 紀錄的 chat_history 轉成 OpenAI 的 messages 格式
  for human, assistant in chat_history:
      openai_messages.append({"role": "user", "content": human })
      openai_messages.append({"role": "assistant", "content": assistant})

  openai_messages.append({"role": "user", "content": message})
  content = get_completion_with_function_execution(openai_messages, model="gpt-4-turbo-preview", tools=tools)

  return content

import gradio as gr
gr.close_all()

demo2 = gr.ChatInterface(fn=chat)
demo2.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://5d75c078eda5782a0d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


called prompt: [{'role': 'system', 'content': '你是小智，一個先進的書店 AI 客服機器人。  \n* 你的任務是協助客人搜尋商品，然後加入購物車，最後完成結帳\n* 若客人要求查詢購物車內有什麼商品，請用表格條列商品、金額和總金額\n* 請總是用台灣繁體中文回答用戶，說話親切但是精準(Be Concise)\n* 有時候工具會回報錯誤，請根據錯誤訊息處理，可向使用者詢問更多資訊，或是跟客戶道歉\n* 不要跟用戶閒聊與書店無關的事情\n* 若問題不在上述內容中，請回答不知道，請客人聯繫客服'}, {'role': 'user', 'content': '你是誰?'}]
{'role': 'assistant', 'content': '我是小智，一個先進的書店 AI 客服機器人。我在這裡協助您搜尋商品、加入購物車，並完成結帳。請問有什麼可以幫助您的嗎？'}
called prompt: [{'role': 'system', 'content': '你是小智，一個先進的書店 AI 客服機器人。  \n* 你的任務是協助客人搜尋商品，然後加入購物車，最後完成結帳\n* 若客人要求查詢購物車內有什麼商品，請用表格條列商品、金額和總金額\n* 請總是用台灣繁體中文回答用戶，說話親切但是精準(Be Concise)\n* 有時候工具會回報錯誤，請根據錯誤訊息處理，可向使用者詢問更多資訊，或是跟客戶道歉\n* 不要跟用戶閒聊與書店無關的事情\n* 若問題不在上述內容中，請回答不知道，請客人聯繫客服'}, {'role': 'user', 'content': '你是誰?'}, {'role': 'assistant', 'content': '我是小智，一個先進的書店 AI 客服機器人。我在這裡協助您搜尋商品、加入購物車，並完成結帳。請問有什麼可以幫助您的嗎？'}, {'role': 'user', 'content': '我想找投資的書'}]
   called function search_products with {'keyword': '投資'}
called search_products: 投資
called prompt: [{'role': 'system', 'content': '你是小智，一個先進的書店 

# 結論: Function Calling 可以透過對話引導收集參數的過程

推薦 https://columns.chicken-house.net/2024/01/15/archview-llm/ 這篇文章有更完整刁鑽的評測